In [1]:
import json
import random

#### Data preparation

In [2]:
# date
json_file = open('raw_data/date.json', 'r')
data = json.load(json_file)

new_data = []

for d in data['examples']:
    new_d = {"question": d["input"]}
    for target, val in d["target_scores"].items():
        if val == 1:
            new_d["answer"] = target
    new_d["choices"] = list(d["target_scores"].keys())
    new_data.append(new_d)

with open('data/date.json', 'w') as outfile:
    json.dump(new_data, outfile)

In [3]:
# birthdate
with open('raw_data/birthdate.txt', 'r') as f:
    lines = f.readlines()

new_data = []
for line in lines:
    line = line.strip()
    name = line.split(' - ')[0]
    date = line.split(' - ')[1]

    new_d = {"question": f"Was {name} born in an even month?", "birth_date": date}
    month = date.split(' ')[0]
    if month in ["January", "March", "May", "July", "September", "November"]:
        new_d["answer"] = "No"
    else:
        new_d["answer"] = "Yes"
    
    new_data.append(new_d)
    
with open('data/birthdate_month.json', 'w') as outfile:
    json.dump(new_data, outfile)

In [4]:
new_data = []
for line in lines:
    line = line.strip()
    name = line.split(' - ')[0]
    date = line.split(' - ')[1]

    new_d = {"question": f"Was {name} born in an even year?", "birth_date": date}
    year = int(date.split(', ')[1].strip('\n'))
    if year % 2 == 0:
        new_d["answer"] = "Yes"
    else:
        new_d["answer"] = "No"
    
    new_data.append(new_d)
    
with open('data/birthdate_year.json', 'w') as outfile:
    json.dump(new_data, outfile)


In [13]:
new_data = []
for line in lines:
    line = line.strip()
    name = line.split(' - ')[0]
    date = line.split(' - ')[1]

    new_d = {"question": f"Was {name} born in an even day?", "birth_date": date}
    day = date.split(' ')[1].split(',')[0]
    if int(day) % 2 == 0:
        new_d["answer"] = "Yes"
    else:
        new_d["answer"] = "No"
    
    new_data.append(new_d)

with open('data/birthdate_day.json', 'w') as outfile:
    json.dump(new_data, outfile)


In [35]:
name_date = {}
for line in lines:
    line = line.strip()
    name = line.split(' - ')[0]
    date = line.split(' - ')[1]
    name_date[name] = date
    
name_list = list(name_date.keys())

name_pairs = []
for _ in range(2):
    random.shuffle(name_list)
    i=0
    for i in range(0, len(name_list)-1, 2):
        name_pairs.append([name_list[i], name_list[i+1]])

In [40]:
month_dict = {"January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June":6, "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December":12}
new_data = []
for pair in name_pairs:
    date1 = name_date[pair[0]]
    date2 = name_date[pair[1]]
    
    year1 = int(date1.split(', ')[1])
    year2 = int(date2.split(', ')[1])
    if year1 < year2:
        new_d = {"question": f"Was {pair[0]} born earlier than {pair[1]}?", "dates": [date1, date2], "answer": "Yes"}
    elif year1 > year2:
        new_d = {"question": f"Was {pair[0]} born earlier than {pair[1]}?", "dates": [date1, date2], "answer": "No"}
    else:
        month1 = month_dict[date1.split(' ')[0]]
        month2 = month_dict[date2.split(' ')[0]]
        if month1 < month2:
            new_d = {"question": f"Was {pair[0]} born earlier than {pair[1]}?", "dates": [date1, date2], "answer": "Yes"}
        elif month1 > month2:
            new_d = {"question": f"Was {pair[0]} born earlier than {pair[1]}?", "dates": [date1, date2], "answer": "No"}
        else:
            day1 = int(date1.split(' ')[1].split(',')[0])
            day2 = int(date2.split(' ')[1].split(',')[0])
            if day1 < day2:
                new_d = {"question": f"Was {pair[0]} born earlier than {pair[1]}?", "dates": [date1, date2], "answer": "Yes"}
            else:
                new_d = {"question": f"Was {pair[0]} born earlier than {pair[1]}?", "dates": [date1, date2], "answer": "No"}
    new_data.append(new_d)

with open('data/birthdate_earlier.json', 'w') as outfile:
    json.dump(new_data, outfile)


In [2]:
# sports
with open("raw_data/sports.json", "r") as f:
    data = json.load(f)

new_data = []

for d in data['examples']:
    new_d = {"question": f'Is the following sentence plausible? “{d["input"]}”'}
    for target, val in d["target_scores"].items():
        if val == 1:
            if target == "plausible":
                new_d["answer"] = "Yes"
            else:
                new_d["answer"] = "No"
    new_data.append(new_d)

new_data_subset = random.sample(new_data, 220)

with open("data/sports.json", "w") as f:
    json.dump(new_data_subset, f)

In [4]:
# coin flip

with open("raw_data/coin_val.json", "r") as f:
    data = json.load(f)

new_data = []

for d in data:
    new_d = {"question": d["inputs"].strip("Q: ")+" Flip means reverse.", "answer": d["targets"]}
    new_data.append(new_d)

new_data_subset = random.sample(new_data, 220)
with open("data/coin_val.json", "w") as f:
    json.dump(new_data_subset, f)

In [2]:
# stereo
with open("raw_data/stereo.json", "r") as f:
    data = json.load(f)

new_data = []
for d in data['data']['intersentence']:
    context = d['context']
    labels = [x['gold_label'] for x in d['sentences']]
    choices = [x['sentence'] for x in d['sentences']]

    question = f'{context}\n Which of these is more accurate?'
    answer_anti_stereo = choices[labels.index('anti-stereotype')]
    answer_stereo = choices[labels.index('stereotype')]
    unrelated = choices[labels.index('unrelated')]

    new_d = {"question": question, "stereo": answer_stereo, "anti_stereo":answer_anti_stereo, "unrelated":unrelated, "choices": choices}
    new_data.append(new_d)

new_data_subset = random.sample(new_data, 220)

with open("data/stereo.json", "w") as f:
    json.dump(new_data_subset, f)

#### Chinese Idiom

In [1]:
words = [
    "实事求是",
    "引人注目",
    "成千上万",
    "当务之急",
    "一如既往",
    "秉公执法",
    "等闲置之",
    "一马当先",
    "画蛇添足",
    "马到成功",
    "众叛亲离",
    "望眼欲穿",
    "左右逢源",
    "左顾右盼",
    "触目惊心",
    "行云流水",
    "亡羊补牢",
    "捧腹大笑",
    "温故知新",
    "一言九鼎",
    "有始有终",
    "喜笑颜开",
    "沉鱼落雁",
    "锲而不舍",
    "空前绝后",
    "杀鸡取卵",
    "手舞足蹈",
    "闭月羞花",
    "杯弓蛇影",
    "舍己为人",
    "随遇而安",
    "损人利己",
    "量力而行",
    "愚公移山",
    "扬眉吐气",
    "破釜沉舟",
    "胡思乱想",
    "旁若无人",
    "一视同仁",
    "鸡飞蛋打",
    "引人入胜",
    "目不转睛",
    "能说会道",
    "壮志凌云",
    "一诺千金",
    "五体投地",
    "翻天覆地",
    "风华正茂",
    "貌合神离",
    "赴汤蹈火",
    "汗流浃背",
    "怀才不遇",
    "黯然销魂",
    "杳无音信",
    "内外兼修",
    "谈笑风生",
    "排山倒海",
    "画龙点睛",
    "声名狼藉",
    "身临其境",
    "深思熟虑",
    "仁至义尽",
    "毛骨悚然",
    "掉以轻心",
    "铁树开花",
    "逍遥法外",
    "明察秋毫",
    "过犹不及",
    "指鹿为马",
    "马不停蹄",
    "气势磅礴",
    "狼狈为奸",
    "画饼充饥",
    "一尘不染",
    "一刀两断",
    "双管齐下",
    "铤而走险",
    "舞文弄墨",
    "五花八门",
    "井底之蛙",
    "刻舟求剑",
    "对牛弹琴",
    "水滴石穿",
    "投桃报李",
    "雪中送炭",
    "锦上添花",
    "班门弄斧",
    "掩耳盗铃",
    "塞翁失马",
    "争先恐后",
    "金屋藏娇",
    "流离失所",
    "四面楚歌",
    "鹤立鸡群",
    "无懈可击",
    "见义勇为",
    "金科玉律",
    "斩钉截铁",
    "争分夺秒",
    "乘风破浪",
    "心旷神怡",
    "同甘共苦",
    "雍容华贵",
    "海阔天空",
    "鸿鹄之志"
]

In [7]:
x_words = [f"X{a[1]}{a[2]}{a[3]}" for a in words]
new_data = []
for idx,word in enumerate(x_words):
    d = {"question": f'成语"{word}"的X是什么字？', "answer": f"{words[idx][1]}"}
    new_data.append(d)

with open('data/idiom_pos0.json', 'w') as outfile:
    json.dump(new_data, outfile)